<a href="https://colab.research.google.com/github/dany-gaga/DGE_analysis/blob/main/Vcf_gff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SNPs_Implies**



In [ ]:
# run this code to mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [ ]:
!pip install gffpandas

In [ ]:
import os
import numpy as np
import pandas as pd
import glob
import gffpandas.gffpandas as gffpd
import itertools
from itertools import chain
from datetime import datetime

In [ ]:
#### convert a gff file into a pandas dataframe

def gff_to_dataframe(file, output):
    """
    This is a function that takes a file name or path and the name of the
    final output to be generated as input
    """
    gff_file = gffpd.read_gff3(file)
    gff_file.to_csv(output)

# gff_to_dataframe('VectorBase-48_AgambiaePEST.gff', 'g.csv')

####

def subset_dataframe(file, sep = None):
    """ This function takes as input,
    a file path and optionaly a seperator, it can be
    '\t', ';', ','. the default being the latter
    """
    file_to_subset = pd.read_csv(file, sep=sep)
    # the values of the first column here is used to subset the data frame
    col_names = list(set(list(file_to_subset.iloc[:,0])))
    for col in col_names:
        t = file_to_subset[file_to_subset.iloc[:,0] == col]
        # here we construct the file names based on the original name with the subsetting string
        t.to_csv(file + str(col) + ".csv", index = False)

####

def list_of_columns(file, start= None, end=None):
    """
    This functions takes as input a file name,
    names of columns to be later combined
    """
    output = []
    for col in (file[start], file[end]):
        col1 = list(col)
        output.append(col1)
    return(output)

def merge_lists(list1, list2, list3=None):
    """
    This function merge into one same list
    elements from two or three different list
    I"""
    if list3 != None:
        merged_list = [(list1[i], list2[i], list3[i]) for i in range(0, len(list1))]
    else:
        merged_list = [(list1[i], list2[i]) for i in range(0, len(list1))]
    return merged_list

###

def merging_vcf_gff(path_vcf, path_gff, types = 'type', the_type='CDS', pos = 'POS', start= 'start', end= 'end'):

    """
    This function takes in multiple input.
    path_vcf = folder in which is located the vcf files
    path_gff = folder in which is located the gff files

    types = type as default, it is the name of the column where the types (genes, cds, intron) are filled.
    the_type= CDS as default. It make you specify on which type you want the merging to be done
    pos = POS as default. It is the column name for POS in a vcf file
    start = start as default. It is the column name for the start of a region in the gff file
    end = end as default. It is the column name for the end of a region in the gff file
    """

    # get path to retrieve the different csv file and sort them all
    vcfs_files = sorted( filter( os.path.isfile, glob.glob(path_vcf + "/*.csv")), reverse=True)
    gffs_files = sorted( filter( os.path.isfile, glob.glob(path_gff + "/*.csv")), reverse=True)

    # create a list of list for both path
    vcf_gff = [vcfs_files, gffs_files]
    # merge files in each lists. The chromosome should match in both the vcf and the gff files
    vgfff = merge_lists(vcf_gff[0], vcf_gff[1])

    # from each tuple present in the merged list,
    # the files will be read
    for vcf1, gff1 in vgfff:

        vcf = pd.read_csv(vcf1, sep=',')
        gff = pd.read_csv(gff1, sep=',')

        # here the types chosen will be used to subset the gff file
        gff = (gff[gff[types] == the_type]).drop_duplicates(subset=['start','end'], keep='first')
        # a list of column names from the vcf and the gff file will be made into one list
        col_names = list(chain.from_iterable(list([vcf.columns, gff.columns])))
        # using the function list_of_columns two list containing each start and end positions will be stored
        cg = list_of_columns(gff, start=start, end=end)
        # the merged list function will be used to merged both lists
        col_gff = merge_lists(cg[0], cg[1])

        both_file =[] # this stores in arrays containg matching line in both the vcf and the gff
        for v in list((vcf[pos])):
            v1 = vcf.loc[vcf[pos] == v].values
            for s,e in col_gff:
                if s <= v <= e:
                    se = gff.loc[(gff[start] == s) & (gff[end] == e)].values
                    break # the break statement here stop the loop once it finda a corresponding line in the gff file
                else:
                    se = np.array([[None] * len(gff.columns)]) # else, it continues and fill the arrays with none

            vcfgff = np.hstack((v1, se)) # here arrays are merged together following the direction vcf + gff

            both_file.append(vcfgff)

        v_g = pd.DataFrame(np.concatenate(both_file)) # A data frame is constructed
        v_g.columns = col_names # using the columns names stored previously, it append it to the dataframe
        v_g['ID']= v_g.attributes.str.slice(3, 13) # can be changed based gene ID length
        #vcfgff['chromosome']= vcfgff['#CHROM'].str[7::1] good to know
        v_g['chrom_pos'] = v_g['#CHROM'].astype(str).str.cat(v_g.POS.astype(str), sep=':')
        v_g = (v_g[v_g[types] == the_type])
        v_g.to_csv(vcf1 + ".csv", index = False) # resulting files are saved in the output folder

# Get CSV files list from a folder
# path = './gffs/'
# path1 = './vcfs/'
# merging_vcf_gff(path1, path)

In [ ]:
fn = '/content/gdrive/MyDrive/vcf_gff/VectorBase-48_AgambiaePEST.gff'
gff_to_dataframe(fn, 'agam_annotation.csv')

In [ ]:
# vcf have been previoulsy converted to csv
# vcf-to-tab < all_vcf.vcf >  all_vcf.csv

vcf = '/content/gdrive/MyDrive/vcf_gff/all_vcf.csv'
gff = '/content/gdrive/MyDrive/vcf_gff/agam_annotation.csv'

In [ ]:
# make three distinct folder for the vcf and gff files to be subsetted and one for the merged files
vcfs = "/content/gdrive/MyDrive/vcf_gff/vcfs"
gffs = "/content/gdrive/MyDrive/vcf_gff/gffs"
v_g = "/content/gdrive/MyDrive/vcf_gff/v_g"
per_samples = "/content/gdrive/MyDrive/vcf_gff/per_samples"

In [ ]:
os.mkdir(vcfs)
os.mkdir(gffs)
os.mkdir(v_g)
os.mkdir(per_samples)

In [ ]:
subset_dataframe(gff, sep=',')

In [ ]:
subset_dataframe(vcf, sep='\t')

In [ ]:
!mv agam_annotation.csvAgam* /content/gdrive/MyDrive/vcf_gff/gffs

In [ ]:
!mv all_vcf.csvAgam* /content/gdrive/MyDrive/vcf_gff/vcfs

In [ ]:
# here, make sure you do not delete the main vcf file
!rm all_vcf.csvAA*

In [ ]:
merging_vcf_gff(vcfs, gffs)

In [ ]:
%cd /content/gdrive/MyDrive/vcf_gff/vcfs/

In [ ]:
!mv *csv.csv /content/gdrive/MyDrive/vcf_gff/v_g

In [ ]:
%cd /content/gdrive/MyDrive/vcf_gff/v_g/

/content/gdrive/MyDrive/vcf_gff/v_g


In [ ]:
!awk '(NR == 1) || (FNR > 1)' *.csv > all_vcf_AgamP4.csv

In [ ]:
!mv all_vcf_AgamP4.csv /content/gdrive/MyDrive/vcf_gff/per_samples

In [ ]:
!pwd

/content


In [ ]:
%cd /content/gdrive/MyDrive/vcf_gff/per_samples/

/content/gdrive/MyDrive/vcf_gff/per_samples


In [ ]:
all_samples = pd.read_csv('all_vcf_AgamP4.csv')
all_samples

In [ ]:
all_samples.columns = all_samples.columns.str.replace('_l1l2_mq10.srt.bam', '')
all_samples.columns

<ipython-input-17-24216463e1cc>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  all_samples.columns = all_samples.columns.str.replace('_l1l2_mq10.srt.bam', '')


Index(['#CHROM', 'POS', 'REF', 'BD01_S9', 'BD02_S8', 'BD03_S7', 'BP02_S9',
       'BP03_S8', 'BP04_S7', 'BU01_S6', 'BU02_S5', 'BU03_S4', 'DA01_S9',
       'DA02_S8', 'DA03_S7', 'DD01_S6', 'DD02_S5', 'DD03_S4', 'DP01_S6',
       'DP02_S5', 'DP03_S4', 'DU01_S3', 'DU02_S2', 'DU03_S1', 'KIS01_S3',
       'KIS02_S2', 'KIS03_S1', 'KIS10_S1', 'KIS6_S3', 'KIS9_S2', 'seq_id',
       'source', 'type', 'start', 'end', 'score', 'strand', 'phase',
       'attributes', 'ID', 'chrom_pos'],
      dtype='object')

In [ ]:
samples = ['BD01_S9', 'BD02_S8', 'BD03_S7', 'BP02_S9',
       'BP03_S8', 'BP04_S7', 'BU01_S6', 'BU02_S5', 'BU03_S4', 'DA01_S9',
       'DA02_S8', 'DA03_S7', 'DD01_S6', 'DD02_S5', 'DD03_S4', 'DP01_S6',
       'DP02_S5', 'DP03_S4', 'DU01_S3', 'DU02_S2', 'DU03_S1', 'KIS01_S3',
       'KIS02_S2', 'KIS03_S1', 'KIS10_S1', 'KIS6_S3', 'KIS9_S2']

for s in samples:
    s_data = all_samples[['#CHROM', 'POS', 'REF', s, 'start', 'end', 'attributes', 'ID', 'chrom_pos']]
    change = str('./.')
    s_data[s].replace(change, value='NA', inplace=True)
    s_data['allele_change'] = s_data['REF'] + '/' + s_data.iloc[:,3].str.split('/').str[1]
    s_data['chrom_pos_mut'] = s_data[['chrom_pos', 'allele_change']].astype(str).agg(':'.join, axis=1)
    s_data.to_csv(s + ".csv", index = False)

In [ ]:
!mv all_vcf_AgamP4.csv /content/gdrive/MyDrive/vcf_gff/v_g

In [ ]:
# get the gff and the reference genome file

!wget https://vectorbase.org/common/downloads/release-55/AgambiaePEST/gff/data/VectorBase-55_AgambiaePEST.gff
!wget https://vectorbase.org/common/downloads/release-55/AgambiaePEST/fasta/data/VectorBase-55_AgambiaePEST_Genome.fasta

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
# install R packages using the R_packages to_be_installed_permanently.ipynb
.libPaths("library")
install.packages("BiocManager")

In [ ]:
%%R
.libPaths("library")
BiocManager::install(c("Biostrings", "ape", "plyr"))

In [ ]:
%%R
.libPaths("library")
if (packageVersion("devtools") < 1.6) {
  install.packages("devtools")
}
devtools::install_github("hadley/lazyeval")
devtools::install_github("hadley/dplyr")

In [ ]:
%%R
.libPaths("library")
install.packages("data.table", type = "source",
    repos = "https://Rdatatable.gitlab.io/data.table")

In [ ]:
%%R

#Loading library
library(ape)
library(Biostrings)
library(plyr)
library(data.table)
library(dplyr)


In [ ]:
%%R

############################
# SETTING UP THE FUNCTIONS #
############################

# The user doesn't need to do anything in this section. We're just creating the functions that we will need.

# Let's write a function to extract useful information from a gene of interest
extract.gene.info <- function(agap.number, gff.table, isoform = 'A'){
  if (!(isoform %in% LETTERS))
    stop('"isoform" should be a single capitalised alphabetical letter')
  isoform.num <- which(LETTERS == isoform)
  gff.focus <- subset(gff.table, grepl(agap.number, attributes) & (!grepl(paste(agap.number, '-[PR][', paste(LETTERS[-isoform.num], collapse = ''), ']', sep = ''), attributes)))
  # Get the vector of indices that will reconstitute the gene sequence
  cds.ranges <- subset(gff.focus, type == 'CDS')[, c('start', 'end')]
  # Get the strand direction for the gene and do a sanity check that only one strand is obtained
  strand <- unique(as.character(gff.focus$strand))
  if (length(strand) > 1) stop('More than one strand direction detected')
  # Same for chromosome
  chrom <- unique(as.character(gff.focus$seqid))
  if (length(chrom) != 1) stop('The gene should be found on one and only one chromosome.')
  # Get the sequence for each of the CDS segments.
  sequence.list <- apply(cds.ranges, 1, function(x) substr(genome[[chrom]], x[1], x[2]))
  # c() can be used to paste the sequences together, but this doesn't seem to work with do.call (which just
  # concatenates everything like the normal c() behaviour), so I don't know how to combine everything together
  # except with this clusmy for loop.
  full.cds.sequence <- sequence.list[[1]]
  for (s in sequence.list[2:length(sequence.list)]){
    full.cds.sequence <- c(full.cds.sequence, s)
  }
  # If necessary, reverse commplement the sequence
  if (strand == '-')
    full.cds.sequence <- reverseComplement(full.cds.sequence)
  # Translate the sequence
  full.aa.sequence <- translate(full.cds.sequence)
  # Split the cds into codons.
  full.codon.list <- sapply(seq(1, length(full.cds.sequence), by = 3), function(i) substr(full.cds.sequence, i, i+2))
  # Translate each of those codons
  full.aa.list <- sapply(full.codon.list, translate, no.init.codon = T)
  # And use that to translate the sequence. This is redundant as we've already done it, but it's just a sanity
  # check.
  full.aa.sequence.2 <- do.call(c, full.aa.list)
  if (full.aa.sequence != full.aa.sequence.2) stop('Codon-by-codon translation didn\'t match full sequence translation.')

  # For a given SNP genomic position, we need to work out in which part of the sequence it is and, if it's a CDS,
  # where on the CDS it sits
  gene.range <- c(min(gff.focus$start), max(gff.focus$end))
  sequence.indices <- rep('intron', gene.range[2] - gene.range[1] + 1)
  for (i in 1:nrow(gff.focus)){
    feature.type <- as.character(gff.focus[i, 'type'])
    if (feature.type %in% c('five_prime_UTR', 'CDS', 'three_prime_UTR'))
      sequence.indices[(gff.focus[i, 'start']:gff.focus[i, 'end']) - gene.range[1] + 1] <- feature.type
  }
  names(sequence.indices) <- as.character(gene.range[1]:gene.range[2])

  list(agap = agap.number,
       sequence.indices = sequence.indices,
       cds.ranges = cds.ranges,
       strand = strand,
       # To save on memory, store the CDS as character, not DNAStringSet
       full.codon.list = sapply(full.codon.list, as.character),
       full.aa.list = sapply(full.aa.list, as.character))
}

# Write a function that will assess the relevance of a SNP, given it's genomic position
assess.SNP <- function(snp, gene.data){
  snp.id <- strsplit(snp, ':')[[1]]
  genome.position <- snp.id[2]
  element.type <- gene.data$sequence.indices[genome.position]
  if (is.na(element.type))
    output <- 'outside'
  else if (element.type != 'CDS')
    output <- as.character(element.type)
  else{
    # Which nucleotide of the sequence is it from
    cds.indices <- unlist(apply(gene.data$cds.ranges, 1, function(x) x[1]:x[2]))
    if (gene.data$strand == '-')
      cds.indices <- rev(cds.indices)
    nuc.index <- which(cds.indices == genome.position)
    codon.index <- ((as.numeric(nuc.index) - 1) %/% 3) + 1
    within.codon.position <- ((as.numeric(nuc.index) - 1) %% 3) + 1
    ref.codon <- gene.data$full.codon.list[[codon.index]]
    ref.aa <- gene.data$full.aa.list[[codon.index]]
    ac <- gene.data$allele.counts[(genome.position), paste('counts', 0:3, sep = '')]
    if (length(snp.id) > 2) {
      alleles = strsplit(snp.id[3], '/')[[1]]
      # If the gene is on the negative strand, need to adjust the coding sequence alleles
      if (gene.data$strand == '-'){
        revcomp.table <- setNames(c('A','C','T','G'), c('T','G','A','C'))
        ref.allele <- revcomp.table[alleles[1]]
        mut.allele <- revcomp.table[alleles[2]]
      }
      else {
        ref.allele <- alleles[1]
        mut.allele <- alleles[2]
      }
      # Sanity check with ref allel
      ref.allele.2 <- substr(ref.codon, within.codon.position, within.codon.position)
      if (ref.allele != ref.allele.2) stop('Reference allele sanity check failed.')
      # Check whether the mutation results in a change in the amino acid
      mut.codon <- replaceAt(DNAString(ref.codon), IRanges(within.codon.position, width = 1), mut.allele)
      mut.aa <- translate(mut.codon, no.init.codon = T)
      if (as.character(mut.aa) == ref.aa)
        output <- list(codon.index, within.codon.position, ref.aa, as.character(mut.aa))
      else
        output <- list(codon.index, within.codon.position, ref.aa, as.character(mut.aa))
    }
    else
      output <- list(codon.index, within.codon.position, NA, NA)
  }
  print(paste(output))
  invisible(output)
}


# If you want to assess more than one SNP at once in a given gene, within a table

assess.snps.table <- function(table, gff){
  table$codon=0
  table$codon_position=0
  table$ref=0
  table$alt=0

  for (i in 1:nrow(table)) {
    if ((is.na(table[i,'allele_change'])) || (nchar(table[i,'allele_change']) != 3)) # avoiding deletions and insertions
      next

    a = try(extract.gene.info(table[i,'ID'], gff), silent = TRUE)
    b = try(assess.SNP(table[i,'chrom_pos_mut'], a), silent = TRUE)
    if (length(b) == 1) {
      table[i,'codon'] = b[[1]]
    } else {
      table[i,'codon'] = b[[1]]
      table[i,'codon_position'] = b[[2]]
      table[i,'ref'] = b[[3]]
      table[i,'alt'] = b[[4]]
    }
  }
  return(table)
}


In [ ]:
%%R
# Load the gff. Put the path to your .gff3 file here.
gff.path <- 'VectorBase-55_AgambiaePEST.gff'
gff <- read.gff(gff.path, GFF3 = T)

# Load the genome. Put the path to your genome here.
genome.path <- 'VectorBase-55_AgambiaePEST_Genome.fasta'
genome <- readDNAStringSet(genome.path)
names(genome) <- sub(' .*', '', names(genome))

In [ ]:
%%R

list.files("/content/gdrive/MyDrive/vcf_gff/per_samples")

fileNames<-Sys.glob("*.csv")

for (file in fileNames) {

  table <- read.csv(file, check.names = F, na.strings=c("","NA"))
  aa_change <- assess.snps.table(table,gff)
  savename <- paste0("aa_changes_", colnames(aa_change)[4], ".csv")
  write.table(aa_change, savename, quote=FALSE, row.names = FALSE, col.names = TRUE, sep='\t')

}